In [1]:
"""機械学習モデルの訓練および、tar.gzからのモデル抽出を行なう。モデルの構造やデータ変換のロジックはmnist_cnn.pyにある。

ref: https://qiita.com/maeda_mikio/items/beefac54c788a7f23218
"""
import os
import pickle

import keras
from keras.datasets import mnist

os.makedirs('data', exist_ok=True)
fout = open('data/mnist.pickle', 'wb')
pickle.dump(mnist.load_data(), fout)

Using TensorFlow backend.



11493376/11490434 [==============================] - 2s 0us/step


In [2]:
import sagemaker
from sagemaker.tensorflow import TensorFlow

session = sagemaker.Session()
bucket_name = session.default_bucket()
inputs = session.upload_data(path='data', bucket=bucket_name, key_prefix='data')

In [4]:
role = sagemaker.get_execution_role()
tf_estimator = TensorFlow(
	entry_point='mnist_cnn.py',
	role=role,
	train_instance_count=1,
	train_instance_type='ml.m5.xlarge',
	framework_version='1.12.0',
	py_version='py3',
	script_mode=True)

tf_estimator.fit(inputs)

train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2022-04-28 05:34:23 Starting - Starting the training job...
2022-04-28 05:34:51 Starting - Preparing the instances for trainingProfilerReport-1651124063: InProgress
......
2022-04-28 05:35:52 Downloading - Downloading input data...
2022-04-28 05:36:13 Training - Training image download completed. Training in progress..2022-04-28 05:36:17,503 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2022-04-28 05:36:17,509 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-04-28 05:36:17,812 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-04-28 05:36:17,831 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-04-28 05:36:17,842 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
    "framework_

In [7]:
import tarfile
import boto3

job_name = tf_estimator.latest_training_job.name

s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)

bucket.download_file(os.path.join(job_name, 'output/output.tar.gz'), 'output.tar.gz')
tarfile.open('output.tar.gz', 'r:gz').extractall()
model = keras.models.load_model('model.h5')

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [8]:
model